# 1. Load cropped

In [ ]:
import os
import glob
import pandas as pd
import random
from PIL import Image
import torch
import torchvision.transforms as transforms
from torch.utils import data
import numpy as np

import math
import numpy as np
import itertools

import torch
from torch import nn as nn
import torch.nn.functional as F
from torch import Tensor
from numpy.matlib import repmat

from models_init import *
from NetBlocks import _CNNBlock,_UnCNNLayer


ModuleNotFoundError: No module named 'models_init'

In [ ]:
import os
import glob
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data

# Paso 1: Definir la función load_cropped
def load_cropped(folder_path, csv_path, patient_list, sample_size=200):
    # Cargar el CSV con pandas
    df = pd.read_csv(csv_path)

    # Convertir el CSV a un diccionario para un acceso rápido
    patient_metadata = {row['CODI']: row['DENSITAT'] for _, row in df.iterrows()}

    # Inicializar la estructura de datos para almacenar los datos de los pacientes seleccionados
    patients_data = []
    images_list = []

    # Iterar sobre cada paciente en la lista de IDs proporcionada
    for patient_id in patient_list:
        # Obtener carpeta del paciente
        patient_folder = glob.glob(os.path.join(folder_path, f"{patient_id}_*"))[0]

        # Verificar que el paciente esté en el CSV
        if patient_id in patient_metadata:
            # Obtener todas las imágenes .png dentro de la carpeta del paciente
            images = glob.glob(os.path.join(patient_folder, "*.png"))

            # Si el paciente tiene imágenes en su carpeta
            if images:
                # Mezclar la lista de imágenes
                random.shuffle(images)

                # Seleccionar una muestra de tamaño sample_size o menos si hay menos imágenes
                images_sampled = random.sample(images, min(sample_size, len(images)))

                for image_path in images_sampled:
                    # Cargar la imagen
                    image = Image.open(image_path)

                    # Convertir a numpy array
                    image_np = np.array(image)

                    # Añadir la imagen a la lista de imágenes
                    images_list.append(image_np)

                # Binariar densidad
                if patient_metadata[patient_id] == "NEGATIVA":
                    dens = 0
                else:
                    dens = 1

                # Añadir la densidad a la lista de metadatos
                patients_data.extend([dens] * len(images_sampled))

    return images_list, patients_data

# Paso 2: Crear la clase Standard_Dataset
class Standard_Dataset(data.Dataset):
    def __init__(self, X, Y=None, transformation=None):
        super().__init__()
        self.X = X
        self.y = Y
        self.transformation = transformation

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        if self.transformation:
            image = self.transformation(image)

        image_tensor = torch.from_numpy(image).float()

        if self.y is not None:
            label = torch.tensor(self.y[idx], dtype=torch.float32)
            return image_tensor, label
        else:
            return image_tensor

# Paso 3: Cargar los datos
folder_path = "C:/Users/mirvi/Desktop/mii/UAB/4.1/PSIV2/detect mateicules/repte3/psiv-repte3/data/Cropped_sample"
csv_path = "C:/Users/mirvi/Desktop/mii/UAB/4.1/PSIV2/detect mateicules/repte3/psiv-repte3/data/PatientDiagnosis.csv"
patient_list = ["B22-25", "B22-29"]
sample_size = 200

images, labels = load_cropped(folder_path, csv_path, patient_list, sample_size)

# Paso 4: Crear el Dataset y DataLoader
dataset = Standard_Dataset(images, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Paso 5: Probar el DataLoader
for batch_idx, (data, target) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}")
    print("Datos (imágenes):", data.shape)
    print("Etiquetas (densidad):", target)
    if batch_idx == 0:
        break  # Solo mostrar el primer batch para verificar que todo funciona


Batch 1
Datos (imágenes): torch.Size([16, 256, 256, 4])
Etiquetas (densidad): tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


## 2. AUTOENCODER

In [ ]:

#  BACKBONE MODULES
class Encoder(nn.Module):
    r"""Encoder class
    `".
    Input Parameters:
        1. inputmodule_params: dictionary with keys ['num_input_channels']
            inputmodule_params['num_input_channels']=Channels of input images
        2. net_params: dictionary defining architecture:
            net_params['block_configs']: list of number of neurons for each
            convolutional block. A block can have more than one layer
            net_params['stride']:list of strides for each block layers
            net_params['drop_rate']: value of the Dropout (equal for all blocks)
        Examples:
            1. Encoder with 4 blocks with one layer each
            net_params['block_configs']=[[32],[64],[128],[256]]
            net_params['stride']=[[2],[2],[2],[2]]
            2. Encoder with 2 blocks with two layers each
            net_params['block_configs']=[[32,32],[64,64]]
            net_params['stride']=[[1,2],[1,2]]

    """

    def __init__(self, inputmodule_params,net_params):
        super().__init__()


        num_input_channels=inputmodule_params['num_input_channels']



        drop_rate=net_params['drop_rate']
        block_configs=net_params['block_configs'].copy()
        n_blocks=len(block_configs)
        if 'stride' in net_params.keys():
            stride=net_params['stride']
        else:
            stride=[]
            for i in np.arange(len(block_configs)):
                stride.append(list(np.ones(len(block_configs[i])-1,dtype=int))+[2])

        # Encoder
        self.encoder=nn.Sequential(
            )
        outchannels_encoder=[]
        for i in np.arange(n_blocks):
            block = _CNNBlock(
                num_input_channels=num_input_channels,
                drop_rate=drop_rate,
                block_config=block_configs[i],
                stride= stride[i]

            )
            self.encoder.add_module("cnnblock%d" % (i + 1), block)

            if stride==1:
                self.encoder.add_module("mxpool%d" % (i + 1),
                                         nn.MaxPool2d(kernel_size=2, stride=2, padding=0))

            num_input_channels=block_configs[i][-1]
           # outchannels_encoder.append(num_input_channels)



    def forward(self, x: Tensor) -> Tensor:

        x=self.encoder(x)

        return x

class Decoder(nn.Module):
    r"""Decoder class
    `".
    Input Parameters:
        1. inputmodule_params: dictionary with keys ['num_input_channels']
            inputmodule_params['num_input_channels']=Channels of input images
        2. net_params: dictionary defining architecture:
            net_params['block_configs']: list of number of neurons for each conv block
            net_params['stride']:list of strides for each block layers
            net_params['drop_rate']: value of the Dropout (equal for all blocks)
    """
    def __init__(self, inputmodule_params,net_params):
        super().__init__()


        num_input_channels=inputmodule_params['num_input_channels']

        self.upPoolMode='bilinear'


        drop_rate=net_params['drop_rate']
        block_configs=net_params['block_configs'].copy()
        self.n_blocks=len(block_configs)

        if 'stride' in net_params.keys():
            stride=net_params['stride']
        else:
            stride=[]
            for i in np.arange(len(block_configs)):
                stride.append(list(np.ones(len(block_configs[i])-1,dtype=int))+[2])


        # Decoder
        self.decoder=nn.Sequential(
            )

        for i0 in np.arange(self.n_blocks)[::-1]:
            i=self.n_blocks-(i0+1)
            block = _CNNBlock(
                num_input_channels=num_input_channels,
                drop_rate=drop_rate,
                block_config=block_configs[i],
                stride=stride[i],
                decoder=True
            )

            # if stride==1:
            #     self.decoder.add_module("uppool%d" % (i + 1),
            #                               nn.Upsample(scale_factor=2,
            #                                           mode=self.upPoolMode, align_corners=True))

            self.decoder.add_module("cnnblock%d" % (i0+1), block)


            num_input_channels=block_configs[i][-1]


        self.decoder[-1][list(self.decoder[-1].keys())[-1]].cnn_layer[2]=nn.Identity()

    def forward(self, x: Tensor) -> Tensor:

        input_sze=x.shape

     #   for i in np.arange(n_blocks)[::-1]:

        x=self.decoder(x)

        return x

##### GENERATIVE MODELS
class AutoEncoderCNN(nn.Module):
    r"""AutoEncoderCNN model class
    `".
    Input Parameters:
        1. inputmodule_paramsEnc: dictionary with keys ['num_input_channels']
            inputmodule_paramsEnc['num_input_channels']=Channels of input images
        2. net_paramsEnc: dictionary defining architecture of the Encoder (see Encoder class)
        3. inputmodule_paramsDec: dictionary with keys ['num_input_channels']
           inputmodule_paramsDec['num_input_channels']=Channels of input images
        4. net_paramsDec: dictionary defining architecture of the Encoder (see Decoder/Encoder classes)
    """

    def __init__(self, inputmodule_paramsEnc,net_paramsEnc,inputmodule_paramsDec,net_paramsDec):
        super().__init__()

        self.inputmodule_paramsEnc=inputmodule_paramsEnc
        self.inputmodule_paramsDec=inputmodule_paramsDec
        self.net_paramsEnc=net_paramsEnc
        self.net_paramsDec=net_paramsDec

        # Encoder
        self.encoder=Encoder(inputmodule_paramsEnc,net_paramsEnc)

        # Decoder
        self.decoder=Decoder(inputmodule_paramsDec,net_paramsDec)

    def forward(self, x: Tensor) -> Tensor:
            # Guardamos el tamaño de la imagen de entrada para usarlo al final (en la fase de upsampling)
            input_size = x.shape

            # Paso por el Encoder
            encoded = self.encoder(x)

            # Paso por el Decoder
            decoded = self.decoder(encoded)

            # Upsampling a las dimensiones originales de la imagen de entrada
            output = F.interpolate(decoded, size=input_size[2:], mode=self.upPoolMode)

            return output


##Annoteted

In [ ]:
import os
import random
import pandas as pd
from glob import glob
from PIL import Image
import numpy as np
import torch
from torch.utils import data
from torch.utils.data import DataLoader

# Función para cargar imágenes y sus rutas desde una estructura de carpetas
def load_images_with_paths(folder_path, excel_path, sample_size=200):
    # Leer el archivo Excel y cargar las columnas especificadas
    df = pd.read_excel(excel_path, usecols=["Pat_ID", "Window_ID", "Presence"])

    # Crear un diccionario para almacenar los datos del Excel
    excel_data = {row['Pat_ID']: {"Window_ID": row['Window_ID'], "Presence": row['Presence']}
                  for _, row in df.iterrows()}

    images_list = []
    paths_list = []

    # Recorrer todas las subcarpetas y cargar imágenes
    for subfolder in glob(os.path.join(folder_path, '**'), recursive=True):
        if os.path.isdir(subfolder):
            images = glob(os.path.join(subfolder, "*.png"))

            if images:
                random.shuffle(images)
                images_sampled = random.sample(images, min(sample_size, len(images)))

                for image_path in images_sampled:
                    image = Image.open(image_path)
                    image_np = np.array(image)

                    # Guardar la imagen y su información de origen
                    images_list.append(image_np)
                    paths_list.append({
                        "folder": os.path.basename(subfolder),  # Solo el nombre de la subcarpeta
                        "image_name": os.path.basename(image_path)  # Nombre del archivo de imagen
                    })

    return images_list, paths_list, excel_data

class Standard_Dataset(data.Dataset):
    def __init__(self, X, paths, Y, transformation=None):
        super().__init__()
        self.X = X
        self.paths = paths
        self.Y = Y
        self.transformation = transformation

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        if self.transformation:
            image = self.transformation(image)

        image_tensor = torch.from_numpy(image).float()
        path_info = self.paths[idx]

        return image_tensor, path_info

# Cargar los datos
folder_path = 'C:/Users/Josep/Documents/DATSETS/Annotated/Annotated'
excel_path = 'C:/Users/Josep/Documents/DATSETS/HP_WSI-CoordAllAnnotatedPatches.xlsx'
sample_size = 200

images, paths, excel_data = load_images_with_paths(folder_path, excel_path, sample_size)

# Crear el Dataset y DataLoader
dataset = Standard_Dataset(images, paths, excel_data)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Probar el DataLoader
for batch_idx, (data, path_info) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}")
    print("Datos (imágenes):", data.shape)
    print("Información de ruta:", path_info)
    if batch_idx == 0:
        break  # Solo muestra el primer batch para verificar que funciona

# Mostrar algunos datos del Excel para verificar
print("\nDatos del Excel:")
for pat_id, info in list(excel_data.items())[:5]:  # Muestra los primeros 5 para verificar
    print(f"PAT_ID: {pat_id}, Window_ID: {info['Window_ID']}, Presence: {info['Presence']}")


Batch 1
Datos (imágenes): torch.Size([16, 256, 256, 4])
Información de ruta: {'folder': ['B22-250_0', 'B22-50_0', 'B22-241_0', 'B22-312_0', 'B22-138_0', 'B22-302_0', 'B22-155_0', 'B22-155_0', 'B22-51_0', 'B22-155_0', 'B22-240_0', 'B22-123_0', 'B22-187_0', 'B22-29_1', 'B22-249_0', 'B22-188_0'], 'image_name': ['02953.png', '00865_Aug5.png', '01489.png', '01154_Aug4.png', '00699_Aug8.png', '00168_Aug8.png', '00065_Aug6.png', '04440_Aug5.png', '00392_Aug4.png', '01518.png', '00500_Aug7.png', '00687_Aug7.png', '00591.png', '00024.png', '01018.png', '00256.png']}

Datos del Excel:
PAT_ID: B22-129, Window_ID: 659, Presence: -1
PAT_ID: B22-68, Window_ID: 352, Presence: -1
PAT_ID: B22-52, Window_ID: 760, Presence: -1
PAT_ID: B22-124, Window_ID: 902_Aug8, Presence: 1
PAT_ID: B22-46, Window_ID: 1340_Aug8, Presence: 1


# Evaluate

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torchvision.utils as vutils
import os

def calculate_psnr(img1, img2):
    """Calcula el PSNR entre dos imágenes de entrada."""
    mse = nn.functional.mse_loss(img1, img2)
    if mse == 0:
        return float('inf')
    return 20 * torch.log10(1.0 / torch.sqrt(mse))

def evaluate_model(model, dataloader, criterion, device="cpu", save_reconstructions=False, save_dir="reconstructions"):
    """
    Evalúa el modelo de autoencoder en el conjunto de datos proporcionado.

    Args:
        model (torch.nn.Module): El modelo a evaluar.
        dataloader (torch.utils.data.DataLoader): DataLoader con los datos de evaluación.
        criterion (torch.nn.Module): Función de pérdida para evaluar el rendimiento.
        device (str): Dispositivo a utilizar ('cpu' o 'cuda').
        save_reconstructions (bool): Si es True, guarda las imágenes originales y reconstruidas.
        save_dir (str): Directorio donde guardar las imágenes reconstruidas.

    Returns:
        dict: Diccionario con métricas de evaluación, incluyendo pérdida promedio y PSNR.
    """
    model.eval()
    total_loss = 0
    total_psnr = 0
    num_batches = 0

    if save_reconstructions and not os.path.exists(save_dir):
        os.makedirs(save_dir)

    with torch.no_grad():
        for batch_idx, (data, _) in enumerate(dataloader):
            data = data.to(device)
            output = model(data)

            # Calcular la pérdida de reconstrucción
            loss = criterion(output, data)
            total_loss += loss.item()

            # Calcular PSNR para el batch
            batch_psnr = calculate_psnr(data, output)
            total_psnr += batch_psnr
            num_batches += 1

            # Guardar imágenes originales y reconstruidas para diagnóstico visual
            if save_reconstructions and batch_idx == 0:  # Solo guardar el primer batch
                comparison = torch.cat([data[:8], output[:8]])  # Concatenar originales y reconstruidas
                vutils.save_image(comparison.cpu(), os.path.join(save_dir, f"reconstruction_batch{batch_idx}.png"), nrow=8)

    avg_loss = total_loss / num_batches
    avg_psnr = total_psnr / num_batches

    return {"average_loss": avg_loss, "average_psnr": avg_psnr}

# Configurar el dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"

# Crear el modelo y cargar al dispositivo
autoencoder = AutoEncoderCNN(
    inputmodule_paramsEnc={'num_input_channels': 4},
    net_paramsEnc={'block_configs': [[32], [64], [128], [256]], 'stride': [[2], [2], [2], [2]], 'drop_rate': 0.1},
    inputmodule_paramsDec={'num_input_channels': 256},
    net_paramsDec={'block_configs': [[128], [64], [32], [4]], 'stride': [[2], [2], [2], [2]], 'drop_rate': 0.1}
)
autoencoder.to(device)

# Crear un DataLoader con datos de evaluación
eval_dataset = Standard_Dataset(images, paths, excel_data)
eval_dataloader = DataLoader(eval_dataset, batch_size=16, shuffle=False)

# Definir la función de pérdida
criterion = nn.MSELoss()

# Evaluar el modelo
metrics = evaluate_model(autoencoder, eval_dataloader, criterion, device=device, save_reconstructions=True, save_dir="reconstructions")

print("Resultados de la evaluación:")
print(metrics)
